In [9]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽고 분할 #############################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 #############################
   
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

## 2.3. 벡터 데이터베이스에 저장  

## 데이터베이스에 저장
##[방법1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )


In [ ]:

##[방법2 ] 로컬에 저장
# database= Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )
# database

In [12]:
## 로컬에 저장된 임베딩 데이터 가져오기

database = Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

database

In [13]:

## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  

## 3.1. 사용자 질문
query = '전세사지 피해자로 인정받기 위한 조건은?.'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=3)

## 4.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 #########

## 4.1. 프롬프트 작성

prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''
## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,)

## 4.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)  
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기 피해자로 인정받기 위해서는 몇 가지 조건을 충족해야 합니다. 다음은 주요 조건들입니다:\n\n1. **주거 조건**: 「주택임대차보호법」 제3조에 따라 주택의 인도와 주민등록을 마치고, 임대차계약증서상의 확정일자를 갖추어야 합니다. 또한, 임차권등기를 마쳤거나 전세권이 설정된 경우도 포함됩니다.\n\n2. **임차보증금 요건**: 임차인의 임차보증금이 5억원 이하이어야 합니다. 특정 조건 하에 2억원의 범위에서 상향 조정이 가능합니다.\n\n3. **임대인의 재정 상황**: 임대인의 파산 또는 회생절차 개시, 임차주택의 경매 또는 공매절차의 시작, 임차인의 집행권원 확보 등이 있어 2인 이상의 임차인이 임차보증금 반환채권의 변제를 받지 못하는 피해가 발생하였거나 예상되어야 합니다.\n\n4. **임대인의 의도성**: 임대인에 대한 수사 개시, 임차보증금을 반환할 능력이 없는 자에 대한 주택 양도 또는 다수 주택 취득 및 임대와 같은 경우, 임차보증금반환채무를 이행하지 않을 의도가 있었다고 의심할 만한 상당한 이유가 있어야 합니다.\n\n다만, 특정 경우에 해당하는 경우 이 조건에서 제외될 수 있습니다. 예를 들어, 보증금 반환을 위한 보증이나 보험에 가입된 경우나, 대항력 또는 우선변제권 등을 통해 자력으로 보증금을 회수할 수 있다고 판단되는 경우 등입니다. \n\n이와 같은 요건들을 모두 충족해야 전세사기 피해자로 인정받을 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 402, 'prompt_tokens': 3078, 'total_tokens': 3480, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_t